In [201]:
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [86]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# split a univariate dataset into train/test sets
def train_test_split(data, test_size):
    """
    Split a univariate dataset into train/test sets
    Arguments:
        data: Observations of time series data as a Pandas DataFrame.
        perc: Percent of data as the test set size.
    Returns:
        (train, test)
    """
    data = data.values
    n = int(len(data) * (1 - test_size))
    return data[:n], data[n:]

# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
    # transform list into array
    train = np.asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict(np.asarray([testX]))
    return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(X_train, X_test, y_train, y_test):
    predictions = list()
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = xgboost_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, -1], predictions

# timer utility function
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
    

In [117]:
# read data
url = "https://raw.githubusercontent.com/ccodwg/Covid19Canada/master/timeseries_prov/cases_timeseries_prov.csv"
covid_df = pd.read_csv(url)
covid_df["date_report"] = pd.to_datetime(covid_df["date_report"], format="%d-%m-%Y")
covid_df.head()

,province,date_report,cases,cumulative_cases
0,Alberta,2020-01-25,0,0
1,Alberta,2020-01-26,0,0
2,Alberta,2020-01-27,0,0
3,Alberta,2020-01-28,0,0
4,Alberta,2020-01-29,0,0


In [124]:
# df.columns = [x.lower() for x in df.columns]
indexer = covid_df[covid_df["province"]=="BC"].index
bc_df = covid_df.loc[indexer, "date_report":"cases"]
bc_df.set_index("date_report", inplace=True)
bc_df.drop(bc_df.columns.difference(["cases"]), 1, inplace=True) # drop all other cols
bc_df = bc_df.asfreq("d")
bc_df.head()

,cases
date_report,
2020-01-25,0
2020-01-26,0
2020-01-27,0
2020-01-28,1
2020-01-29,0


### Create lagged features

In [125]:
def create_lagged_features(df):
    lags = [1, 2, 3, 4, 5, 6, 7]
    for lag in lags:
        df[f"cases(t-{lag})"] = df["cases"].shift(lag)
    df.dropna(inplace=True)
    return df

model_df = create_lagged_features(bc_df)
model_df.head()

,cases,cases(t-1),cases(t-2),cases(t-3),cases(t-4),cases(t-5),cases(t-6),cases(t-7)
date_report,,,,,,,,
2020-02-01,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2020-02-02,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-02-03,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2020-02-04,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2020-02-05,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Split train / test data

In [206]:
# split based on 30 days ago
split_date = datetime.today() - timedelta(days=30)
train_df = model_df[model_df.index <= split_date]
test_df = model_df[model_df.index > split_date]

In [275]:
# A parameter grid for XGBoost
model_params = {
        'min_child_weight': [1],
        'gamma': [1],
        'max_depth': [4]
        }

In [276]:
start_time = timer(None)
param_grid = list(ParameterGrid(model_params))
results = pd.DataFrame()
for i, params in enumerate(param_grid):
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',
                               min_child_weight=params["min_child_weight"],
                               gamma=params["gamma"],
                               max_depth=params["max_depth"],
                              n_estimators=1000)
    predictions = list()
    
    # seed history with training dataset
    history = train_df.copy()
    
    # walk-forward validation
    for j in range(len(test_df)):
        y_test = np.asarray(test_df["cases"])
        y_train = np.asarray(history["cases"])
        X_test = np.asarray(test_df[test_df.columns.difference(["cases"])])
        X_train = np.asarray(history[history.columns.difference(["cases"])])
        
        model = xgb_reg.fit(X_train, y_train)
        
        predictions.append(xgb_reg.predict(np.asarray([X_test[j]])))
        history.append(test_df.iloc[j])
    
    
    mae = mean_absolute_error(y_test, predictions)
    results.loc[i, "MAE"] = mae
    results.loc[i, "min_child_weight"] = params["min_child_weight"]
    results.loc[i, "gamma"] = params["gamma"]
    results.loc[i, "max_depth"] = params["max_depth"]
    
best_params = {
    "min_child_weight": int(results.loc[0, "min_child_weight"]),
    "gamma": int(results.loc[0, "gamma"]),
    "max_depth": int(results.loc[0, "max_depth"])
}
    
results.sort_values("MAE", inplace=True)
timer(start_time)
print(results)


 Time taken: 0 hours 0 minutes and 10.52 seconds.
          MAE  min_child_weight  gamma  max_depth
0  197.357086               1.0    1.0        4.0


In [300]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',
                           n_estimators=1000,
                           **best_params)

best_model = xgb_reg.fit(np.asarray(train_df[train_df.columns.difference(["cases"])]), np.asarray(train_df["cases"]))
forecast = best_model.predict(np.asarray(test_df[test_df.columns.difference(["cases"])]))

In [318]:
type(forecast)

numpy.ndarray

In [328]:
forecast_df = test_df[["cases"]].copy()
forecast_df["y_pred"] = np.round(forecast,0)

In [329]:
forecast_df.head(25)

,cases,y_pred
date_report,,
2021-03-06,0,37.0
2021-03-07,0,11.0
2021-03-08,1462,1644.0
2021-03-09,550,753.0
2021-03-10,531,479.0
2021-03-11,569,511.0
2021-03-12,648,498.0
2021-03-13,0,33.0
2021-03-14,0,7.0
